In [1]:
import numpy as np
from numba import njit

@njit
def check_location(X, Y, defined_domain, x, no_nans_in_domain = False):
    
    Xmax = X[0,-1]
    Xmin = X[0, 0]
    Ymin = Y[0, 0]
    Ymax = Y[-1,0]
        
    xp = x[0]
    yp = x[1]
    
    if no_nans_in_domain:
        
        if Xmin < xp < Xmax and Ymin < yp < Ymax:
            
            loc = "IN"
            
            return loc, None, None
        
        else:
        
            loc = "OUT"
            
            return loc, None, None
 
    else:
        idx_x = np.searchsorted(X[0,:], xp)
        idx_y = np.searchsorted(Y[:,0], yp)
        
        if xp < Xmin or xp > Xmax or yp < Ymin or yp > Ymax or np.isnan(xp) or np.isnan(yp):
            
            loc = "OUT"
        
        else:
            
            if idx_x == 0:
                    
                idx_x = 1
                    
            if idx_y == 0:
                    
                idx_y = 1
            
            Condition_nan = np.sum(defined_domain[idx_y-1:idx_y+1, idx_x-1:idx_x+1].ravel())
        
            if Condition_nan == 4:
        
                loc = "IN"
                    
            elif Condition_nan == 0:
            
                loc = "OUT"
        
            elif Condition_nan == 2:
            
                loc = "BOUNDARY"
                
            elif Condition_nan == 1:
                
                if defined_domain[idx_y-1, idx_x-1]:
                    
                    if idx_y == defined_domain.shape[0]-1 and idx_x == defined_domain.shape[1]-1:
                        
                        loc = "OUT"
                    
                    else:
                        
                        if x[1] + x[0] < Y[idx_y, idx_x-1] + X[idx_y, idx_x-1]:
                        
                            loc = "BOUNDARY"
                        
                        else:
                        
                            loc = "OUT"
                        
                elif defined_domain[idx_y, idx_x]:
                    
                    if idx_y == 1 and idx_x == 1:
                    
                        loc = "OUT"
                    
                    else:
                    
                        if x[1] + x[0] > Y[idx_y, idx_x-1] + X[idx_y, idx_x-1]:
                        
                            loc = "BOUNDARY"
                        
                        else:
                        
                            loc = "OUT"
                        
                elif defined_domain[idx_y, idx_x-1]:
                    
                    if idx_y == 1 and idx_x == defined_domain.shape[1]-1:
                        
                        loc = "OUT"
                    
                    else:
                        
                        if x[1] - x[0] < Y[idx_y-1, idx_x-1] - X[idx_y-1, idx_x-1]:
                        
                            loc = "BOUNDARY"
                        
                        else:
                        
                            loc = "OUT"
                        
                elif defined_domain[idx_y-1, idx_x]:
                    
                    if idx_y == defined_domain.shape[0]-1 and idx_x == 1:
                    
                        loc = "OUT"
                    
                    else:
                    
                        if x[1] - x[0] > Y[idx_y-1, idx_x-1] - X[idx_y-1, idx_x-1]:
                        
                            loc = "BOUNDARY"
                        
                        else:
                        
                            loc = "OUT"
                
            elif Condition_nan == 3:
                
                loc = "BOUNDARY"
         
        return loc, idx_x, idx_y